In [39]:
# nuclio: ignore
import nuclio

## Define environment variables

In [40]:
# nuclio: ignore
from os import environ
version = '1.0'
#environ['MODEL_FILE'] = 'saved_model.pb'.format(version)
environ['MODEL_FILE'] = 'saved_model.pb'

In [41]:
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c MODEL_FILE=${MODEL_FILE}
%nuclio env -c MODEL_FILEPATH=/tmp/mlmodel/${MODEL_FILE}
%nuclio env -l MODEL_FILEPATH=/User/docs/site/en/tutorials/images/saved_models/1551857232/${MODEL_FILE}

%nuclio: setting 'MODEL_FILEPATH' environment variable


In [42]:
%%nuclio cmd -c
pip install --force tensorflow
pip install --force tensorflow_hub
pip install v3io_frames
pip install dask_ml
apt-get update && apt-get install -y wget
mkdir -p /tmp/mlmodel

In [43]:
%nuclio cmd -c wget -O /tmp/mlmodel/${MODEL_FILE} --header "x-v3io-session-key: ${V3IO_ACCESS_KEY}" http://10.96.108.8:8081/users/${V3IO_USERNAME}/demos/netops/models/${MODEL_FILE}

## Set config

In [44]:
%%nuclio config 
spec.triggers.secs.attributes.interval = "60s"
spec.build.baseImage = "python:3.6-jessie"
spec.volumes.volume.name = "fs"
spec.volumes.volume.flexVolume.driver = "v3io/fuse"
spec.volumes.volume.flexVolume.secretRef.name = "jupyter-63mzkiyfdm-pgruh-v3io-fuse"
spec.volumes.volumeMount.volumeMount.name = "fs"
spec.volumes.volumeMount.volumeMount.mountPath = "/v3io"
spec.volumes.volumeMount.volumeMount.readOnly = False

%nuclio: setting spec.triggers.secs.attributes.interval to '60s'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'
%nuclio: setting spec.volumes.volume.name to 'fs'
%nuclio: setting spec.volumes.volume.flexVolume.driver to 'v3io/fuse'
%nuclio: setting spec.volumes.volume.flexVolume.secretRef.name to 'jupyter-63mzkiyfdm-pgruh-v3io-fuse'
%nuclio: setting spec.volumes.volumeMount.volumeMount.name to 'fs'
%nuclio: setting spec.volumes.volumeMount.volumeMount.mountPath to '/v3io'
%nuclio: setting spec.volumes.volumeMount.volumeMount.readOnly to False


## Define function init

In [45]:
import os
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
import v3io_frames as v3f
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define v3io TSDB client
client = v3f.Client('framesd:8081')



# Define parameters

In [46]:
image_size = 160 # All images will be resized to 160x160
IMG_SHAPE = (image_size, image_size, 3)
batch_size = 32
zip_file = tf.keras.utils.get_file(origin="https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip", 
                                   fname="cats_and_dogs_filtered.zip", extract=True)
base_dir, _ = os.path.splitext(zip_file)
validation_dir = os.path.join(base_dir, 'validation')


In [89]:
print (base_dir)
print (validation_dir)

/User/.keras/datasets/cats_and_dogs_filtered
/User/.keras/datasets/cats_and_dogs_filtered/validation


In [50]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.trainable = True
new_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])
new_model.summary()


Consider using a TensorFlow optimizer from `tf.train`.


W0310 02:56:57.688567 140566266135936 network.py:1430] This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.RMSprop object at 0x7fd6827d64e0>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.


W0310 02:56:57.845193 140566266135936 keras_saved_model.py:212] Model was compiled with an optimizer, but the optimizer is not from `tf.train` (e.g. `tf.train.AdagradOptimizer`). Only the serving graph was exported. The train and evaluate graphs were not added to the SavedModel.


INFO:tensorflow:Signatures INCLUDED in export for Classify: None


I0310 02:57:08.973551 140566266135936 export.py:587] Signatures INCLUDED in export for Classify: None


...

INFO:tensorflow:No assets to write.


I0310 02:57:09.721909 140566266135936 builder_impl.py:124] No assets to write.


INFO:tensorflow:SavedModel written to: ./saved_models/1552186589/saved_model.pb


I0310 02:57:09.894405 140566266135936 builder_impl.py:414] SavedModel written to: ./saved_models/1552186589/saved_model.pb


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_9 ( (None, 1280)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________


In [94]:
%%nuclio handler
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow validation images in batches of 20 using test_datagen generator
prediction = datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

for item in prediction.classes:
    print('Image is a ' + {False: 'Cat', True :'Dog'}[item])
    
    
    

Found 1000 images belonging to 2 classes.
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Cat
Image is a Ca

In [ ]:
# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
#print ('Total validation cat images:', len(os.listdir(validation_cats_dir)))

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
#print ('Total validation dog images:', len(os.listdir(validation_dogs_dir)))

************

In [ ]:



validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

for item in validation_generator.classes:
    print('Image is a ' + {False: 'Cat', True :'Dog'}[item])

In [93]:
%nuclio deploy -n catsanddogs -p test1 -v

%nuclio: cannot deploy
%nuclio: usage: nuclio deploy [-h] [--dashboard-url DASHBOARD_URL] [--project PROJECT]
                     [--verbose]
                     notebook
nuclio deploy: error: argument notebook: can't open 'catsanddogs': [Errno 2] No such file or directory: 'catsanddogs'


In [91]:
# nuclio: ignore
nuclio.magic.print_handler_code()

%nuclio: notebook exported to /tmp/nuclio-handler-25dadkxd
# Generated by nuclio.export.NuclioExporter on 2019-03-10 05:40

import os
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
import v3io_frames as v3f
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define v3io TSDB client
client = v3f.Client('framesd:8081')

image_size = 160 # All images will be resized to 160x160
IMG_SHAPE = (image_size, image_size, 3)
batch_size = 32
zip_file = tf.keras.utils.get_file(origin="https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip", 
                                   fname="cats_and_dogs_filtered.zip", extract=True)
base_dir, _ = os.path.splitext(zip_file)
validation_dir = os.path.join(base_dir, 'validation')

print (base_dir)
print (validation_dir)

saved